#eXamine tutorial (with automation)

Trying to recreate the eXamine tutorial with automation ... 
Let's see how this goes

In [ ]:
# HTTP Client for Python
import requests

# Standard JSON library, currently unused
import json

# Basic Setup
PORT_NUMBER = 1234 #Yes, this is really the Cytoscape Basic Port

#The Base path for the CyRest API
BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'

#Helper command to call a command via HTTP GET
def executeRestCommand(namespace="",command="",args={}):
    postString = BASE + "commands/"+namespace+"/"+command
    firstarg = True;
    for arg in args:
        postString+=("?" if firstarg else "&")
        postString+=arg+"="+args[arg]
        if (firstarg):
            firstarg = False;
            
    print(postString)
    res = requests.get(postString)
    return res

print("test")
    


In [9]:
#First we import our demo network
res = executeRestCommand("network","import url",{"indexColumnSourceInteraction":"1","indexColumnTargetInteraction":"2","url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/edges.txt"})
print(res.text)


http://localhost:1234/v1/commands/network/import url?indexColumnTargetInteraction=2&url=https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/edges.txt&indexColumnSourceInteraction=1


DGraphView: suid=99573, model=edges.txt
Finished



In [36]:
#TODO: This can surely be done more elegant
networkSUID = res.text.split("suid=",1)[1].split(",")[0];
print(networkSUID)

99573


In [37]:
#Next we import node annotations
res = executeRestCommand(
    "table",
    "import url",
    {"firstRowAsColumnNames":"true",
     "keyColumnIndex" : "1",
     "startLoadRow" : "1",
     "dataTypeList":"s,s,f,f,f,s,s,s,sl,sl,sl,sl",
     "url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/nodes_induced.txt"})

print(res.text)

http://localhost:1234/v1/commands/table/import url?dataTypeList=s,s,f,f,f,s,s,s,sl,sl,sl,sl&firstRowAsColumnNames=true&keyColumnIndex=1&startLoadRow=1&url=https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/nodes_induced.txt


Mapped to tables:
   edges.txt root shared  node

Finished



In [38]:
#We select all nodes with the value "small" in the column "Module"

res = executeRestCommand(
    "network",
    "select",
    {"nodeList":"Module:small"})

http://localhost:1234/v1/commands/network/select?nodeList=Module:small


In [39]:
#As a next step we generate the groups via eXamine
res = executeRestCommand(
    "examine",
    "generate groups",
    {"selectedGroupColumns":"Process,Function,Component,Pathway"})

print(res.text)

http://localhost:1234/v1/commands/examine/generate groups?selectedGroupColumns=Process,Function,Component,Pathway


Finished



In [40]:
#Ok, time to enrich our newly greated group nodes with some interesting annotations
res = executeRestCommand(
    "table",
    "import url",
    {"firstRowAsColumnNames":"true",
     "keyColumnIndex" : "1",
     "startLoadRow" : "1",
     "url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/sets_induced.txt"})

print(res.text)

http://localhost:1234/v1/commands/table/import url?url=https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/sets_induced.txt&firstRowAsColumnNames=true&keyColumnIndex=1&startLoadRow=1


Mapped to tables:
   edges.txt root shared  node

Finished



In [14]:
# Adjust the visualization settings
res = executeRestCommand(
    "examine",
    "update settings",
    {"labelColumn": "Symbol",
     "urlColumn": "URL",
     "scoreColumn": "Score",
     "showScore": "true"})

print(res.text)

http://localhost:1234/v1/commands/examine/update settings?urlColumn=URL&labelColumn=Symbol&showScore=true&scoreColumn=Score
Finished



In [15]:
# Launch the interactive eXamine visualization
res = executeRestCommand(
    "examine",
    "interact",
    {})

print(res.text)

http://localhost:1234/v1/commands/examine/interact
Finished

